In [18]:
from __future__ import print_function
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
import csv
import os

In [2]:
# load and use credentials
with open("client_credentials.json", "r") as f:
    credentials = json.load(f)
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=credentials["client_id"], client_secret=credentials["client_secret"]))

In [3]:
# read playlist ids
with open("playlist_ids.json", "r") as f:
    playlist_ids = json.load(f)

In [21]:
multiclass_labels = {'rock': 1, 'pop': 2, 'hiphop': 3, 'blues': 4, 'electronic': 5, 'jazz': 6, 'classical': 7, 'disco': 8, 'folk': 9, 'alternative': 10}
fieldnames = ['id','track_name', 'artist_name', 'preview_url', 'danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','duration_ms','time_signature']

file_exists = os.path.isfile('data.csv')

for genre, playlist_id in playlist_ids.items():
    items = spotify.playlist_tracks(playlist_id, fields='items.track.id, items.track.name, items.track.artists, items.track.preview_url, total', limit=50, offset=0)['items']
    track_ids = [item['track']['id'] for item in items if item['track'] is not None]
    meta_data = [{'id': item['track']['id'], 'track_name': item['track']['name'], 'artist_name': item['track']['artists'][int(0)]['name'], 'preview_url': item['track']['preview_url']} for item in items if item['track'] is not None]    
    features = spotify.audio_features(track_ids)

    with open('data.csv', 'a') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames + ['multiclass_label'])
        if not file_exists:
            writer.writeheader()  # file doesn't exist yet, write a header

        for meta_data, features in zip(meta_data, features):
            meta_data.update(features)
            track = {field: meta_data[field] for field in fieldnames}
            track['multiclass_label'] = multiclass_labels[genre]
            writer.writerow(track)

